<a href="https://colab.research.google.com/github/j54854/myColab/blob/main/pom2_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 生産管理技術2_9の補助資料
この補助資料では、生産管理技術2の講義で扱ったジョブショップスケジューリング問題を、PuLP（とフリーのソルバーCBC）を用いて解く方法を学ぶ。

## 簡単な数値例

まず、講義内で取り上げた、ジョブショップスケジューリング問題の簡単な数値例を再掲しておこう（Pythonの都合でジョブ番号、工程番号、機械番号が 0 から始まるように変更されている点に注意）。

## 加工経路

| | 第0工程 | 第1工程 | 第2工程 |
| ---- | :----: | :----: | :----: |
| J0 | M2 | M0 | M1 |
| J1 | M0 | M2 | M1 |
| J2 | M1 | M2 | M0 |

## 加工時間


| | 第0工程 | 第1工程 | 第2工程 |
| ---- | ---- | ---- | ---- |
| J0 | 30 | 70 | 20 |
| J1 | 40 | 40 | 60 |
| J2 | 20 | 50 | 80 |



最初に、このスケジューリング問題の情報をPythonで扱えるデータとして表現しておく（簡単のため、それぞれ独立な2次元リストとした）。ジョブ番号を j、工程番号を k とすると、これで、オペレーション (j, k) を処理する機械の番号を pr[j][k] で、それかかる加工時間を pt[j][k] で、それぞれ参照できるようになった。また、ジョブ数を n_job に、機械数（機械番号の最大値 +1）を n_mac に、それぞれ代入してある。

In [ ]:
pr = [  # processing routes: pr[j][k]
    [2, 0, 1],
    [0, 2, 1],
    [1, 2, 0]
]

pt = [  # processing times: pt[j][k]
    [30, 70, 20],
    [40, 40, 60],
    [20, 50, 80]
]

n_job = len(pr)  # number of jobs
n_mac = max(sum(pr, [])) +1  # number of machines (A bit tricky, but sum(pr, []) flattens pr.)

print('There are {} jobs and {} machines.'.format(n_job, n_mac))

There are 3 jobs and 3 machines.


今後、必要に応じて、参照したいオペレーション (j, k) をすぐに見つけられるように、ここで、オペレーションのリストをいくつか定義しておこう。op_all は、すべてのオペレーションを含むリスト、op_j[j] は、ジョブ j を構成するオペレーションのリスト、op_m[m] は機械 m で処理されるオペレーションのリストである。

In [ ]:
op_all = []  # list of all operations
op_j = [[] for _ in range(n_job)]  # operations of each job
op_m = [[] for _ in range(n_mac)]  # operations on each machine

for j in range(n_job):
  for k in range(len(pr[j])):
    op_all.append((j, k))
    op_j[j].append((j, k))
    op_m[pr[j][k]].append((j, k))

print('All operations:', op_all)
for j in range(n_job):
  print('Operations of job {}:'.format(j), op_j[j])
for m in range(n_mac):
  print('Operations on machine {}:'.format(m), op_m[m])

All operations: [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
Operations of job 0: [(0, 0), (0, 1), (0, 2)]
Operations of job 1: [(1, 0), (1, 1), (1, 2)]
Operations of job 2: [(2, 0), (2, 1), (2, 2)]
Operations on machine 0: [(0, 1), (1, 0), (2, 2)]
Operations on machine 1: [(0, 2), (1, 2), (2, 0)]
Operations on machine 2: [(0, 0), (1, 1), (2, 1)]


ジョブショップスケジューリング問題では、2つのオペレーション (i, k) と (j, l) が同じ機械上で処理される場合、それらの順序を指定する必要があった。ここで、順序を指定する必要のあるオペレーションのペアを表す (i, k, j, l) のリストも作っておこう。

In [ ]:
x_keys = []  #　list of (i, k, j, l) to be considered

for ops in op_m:
  for n, (i, k) in enumerate(ops):
    for (j, l) in ops[n +1:]:
      x_keys.append((i, k, j, l))

print(x_keys)

[(0, 1, 1, 0), (0, 1, 2, 2), (1, 0, 2, 2), (0, 2, 1, 2), (0, 2, 2, 0), (1, 2, 2, 0), (0, 0, 1, 1), (0, 0, 2, 1), (1, 1, 2, 1)]


続いて、この問題をPuLPを使って解いていく。PuLPを利用できるようにするために、pipでPuLPをインストールしてから、それをインポートする。


In [ ]:
!pip install pulp
import pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 71.1 MB/s eta 0:00:00


次に、PuLPで問題（mip）を定義して、その問題（mip）に決定変数と目的関数を登録する。

In [ ]:
mip = pulp.LpProblem('job_shop', sense=pulp.LpMinimize)  # minimization problem
c_max = pulp.LpVariable('c_max', 0, None)  # makespan
st = pulp.LpVariable.dicts('st', op_all, 0, None)  # starting times
x = pulp.LpVariable.dicts('x', x_keys, cat=pulp.LpBinary)  # binary (0-1) variables
mip += c_max  # objective function

続いて、制約条件を登録していく。まず、各ジョブの加工経路によって指定される、同一ジョブのオペレーション同士の順序関係の制約条件と，メイクスパンが各ジョブの最終オペレーションの終了時刻以降になる制約
条件を追加する。

In [ ]:
for ops in op_j:
  for (j, k) in ops[:-1]:  # except for the last one
    mip += st[(j, k +1)] -st[(j, k)] >= pt[j][k]
  j, k = ops[-1] # the last operation
  mip += c_max -st[(j, k)] >= pt[j][k]

次に、同一機械上で処理されるオペレーション間の制約条件を登録していく。この制約条件を線形式で表すために、Big-Mを使う。Big-Mの値が大きすぎると計算効率が落ちることがあるので、最初にBig-Mの値を「全オペレーションの加工時間の総和+1」に設定していることにも注意してほしい。

In [ ]:
big_M = sum([pt[j][k] for (j, k) in op_all]) +1

for i, k, j, l in x_keys:
  mip += st[(i, k)] -st[(j, l)] +big_M * x[(i, k, j, l)] >= pt[j][l]
  mip += st[(j, l)] -st[(i, k)] +big_M * (1 -x[(i, k, j, l)]) >= pt[i][k]

これで問題を登録できた。念のため、登録した問題を確認しておこう。

In [ ]:
print(mip)

これで準備が整ったので、最後に、ソルバーで解を導出する。mip.solve()は、求解のためのメソッドの呼出しである。その返り値statusがOptimalになっていれば最適解が得られていることがわかる。

In [ ]:
status = mip.solve()
print(pulp.LpStatus[status])

最適解（における各オペレーションの開始時刻）とそのときの目的関数値（メイクスパン）は次のようにして確認すればよい。

In [ ]:
print('Starting times:')
for (j, k) in op_all:
  print('st[({}, {})] = {}'.format(j, k, st[(j, k)].value()))
print('Objective function value: c_max = {}'.format(mip.objective.value()))


以上のように、pythonのPuLPライブラリ（とそれに付随しているフリーのソルバーCBC）を使うと、（0-1混合整数線形計画問題として定式化した） ジョブショップスケジューリング問題の解を求めることができる。